In [1]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 91.7 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 105.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is t

## Local Inference on GPU 
Model page: https://huggingface.co/microsoft/deberta-v3-base

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/microsoft/deberta-v3-base)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="microsoft/deberta-v3-base")

2025-10-13 10:32:59.187105: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760351579.351471      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760351579.399774      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cuda:0


In [5]:
# Load model directly
from transformers import AutoModel
model = AutoModel.from_pretrained("microsoft/deberta-v3-base", dtype="auto")

## Remote Inference via Inference Providers 
Ensure you have a valid **HF_TOKEN** set in your environment. You can get your token from [your settings page](https://huggingface.co/settings/tokens). Note: running this may incur charges above the free tier.
The following Python example shows how to run the model remotely on HF Inference Providers, automatically selecting an available inference provider for you. 
For more information on how to use the Inference Providers, please refer to our [documentation and guides](https://huggingface.co/docs/inference-providers/en/index).

In [6]:
import os
os.environ['HF_TOKEN'] = '/'

In [11]:
# =====================================
# MINIMAL FIXED VERSION WITH VALIDATION
# 60K TRAIN + 15K VALIDATION - FIXED
# =====================================

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
import re
import gc
import os
import joblib

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# =====================================
# FIX: Replace lambda with proper function
# =====================================

class TargetTransformer:
    def __init__(self):
        self.scaler = RobustScaler()
        self.is_fitted = False
    
    def fit_transform(self, targets):
        targets = np.array(targets)
        log_targets = np.log1p(targets)
        scaled = self.scaler.fit_transform(log_targets.reshape(-1, 1)).flatten()
        self.is_fitted = True
        return scaled
    
    def transform(self, targets):  # ✅ ADDED MISSING METHOD
        if not self.is_fitted:
            raise ValueError("Transformer not fitted")
        targets = np.array(targets)
        log_targets = np.log1p(targets)
        return self.scaler.transform(log_targets.reshape(-1, 1)).flatten()
    
    def inverse_transform(self, scaled_targets):
        if not self.is_fitted:
            raise ValueError("Transformer not fitted")
        log_targets = self.scaler.inverse_transform(scaled_targets.reshape(-1, 1)).flatten()
        return np.expm1(log_targets)

# =====================================
# SMAPE CALCULATION FOR VALIDATION
# =====================================

def calculate_smape(preds, targets):
    """Calculate SMAPE metric for validation"""
    return 100 * np.mean(2 * np.abs(preds - targets) / (np.abs(preds) + np.abs(targets) + 1e-8))

# =====================================
# ORIGINAL TEXT PROCESSING (WORKING)
# =====================================

def fast_text_processing(text):
    """Fast but effective text processing"""
    if pd.isna(text):
        return "No description"
    
    text = str(text)
    parts = []
    
    # 1. Product Name (CRITICAL)
    name_match = re.search(r'Item Name:\s*([^\n]+)', text, re.IGNORECASE)
    if name_match:
        parts.append(f"PRODUCT: {name_match.group(1).strip()}")
    
    # 2. Quantity & Unit (CRITICAL)
    value_match = re.search(r'Value:\s*([\d.]+)', text, re.IGNORECASE)
    unit_match = re.search(r'Unit:\s*([^\n]+)', text, re.IGNORECASE)
    if value_match and unit_match:
        parts.append(f"SIZE: {value_match.group(1)} {unit_match.group(1)}")
    
    # 3. First 3 Bullet Points (IMPORTANT)
    bullets = re.findall(r'Bullet Point \d+:\s*([^\n]+)', text)[:3]
    for bullet in bullets:
        parts.append(f"FEAT: {bullet.strip()}")
    
    # Fallback
    if not parts:
        text = re.sub(r'\s+', ' ', text)
        parts.append(text[:300])
    
    return " | ".join(parts)

# =====================================
# ORIGINAL FEATURE ENGINEERING (WORKING)
# =====================================

def extract_essential_features(df):
    """Only the most important features"""
    features = []
    
    for text in df['catalog_content']:
        text = str(text)
        feature_row = []
        
        # 1. EXTRACTED VALUE (MOST IMPORTANT)
        value_match = re.search(r'Value:\s*([\d.]+)', text, re.IGNORECASE)
        value = float(value_match.group(1)) if value_match else 0.0
        feature_row.append(value)
        
        # 2. UNIT TYPE (VERY IMPORTANT)
        unit_match = re.search(r'Unit:\s*([^\n]+)', text, re.IGNORECASE)
        unit_text = unit_match.group(1).lower() if unit_match else ""
        
        # Only essential unit types
        unit_ounce = 1 if any(word in unit_text for word in ['ounce', 'oz']) else 0
        unit_count = 1 if any(word in unit_text for word in ['count', 'ct', 'piece']) else 0
        unit_fluid = 1 if any(word in unit_text for word in ['fluid', 'fl']) else 0
        
        feature_row.extend([unit_ounce, unit_count, unit_fluid])
        
        # 3. TEXT LENGTH (IMPORTANT)
        feature_row.append(len(text))
        feature_row.append(len(re.findall(r'Bullet Point \d+:', text)))
        
        features.append(feature_row)
    
    return np.array(features)

# =====================================
# ORIGINAL MODEL (WORKING)
# =====================================

class FastDebertaPredictor(nn.Module):
    def __init__(self, model_name="microsoft/deberta-v3-base", n_features=6, dropout=0.3):
        super().__init__()
        
        # DeBERTa v3 base
        self.deberta = AutoModel.from_pretrained(model_name)
        
        # ORIGINAL FINE-TUNING: Unfreeze last 4 layers
        for param in self.deberta.parameters():
            param.requires_grad = False
            
        for layer in self.deberta.encoder.layer[-4:]:
            for param in layer.parameters():
                param.requires_grad = True
        
        # Efficient regressor
        self.regressor = nn.Sequential(
            nn.Linear(768 + n_features, 384),
            nn.ReLU(),
            nn.Dropout(dropout),
            
            nn.Linear(384, 192),
            nn.ReLU(),
            nn.Dropout(dropout),
            
            nn.Linear(192, 96),
            nn.ReLU(),
            
            nn.Linear(96, 1)
        )
    
    def forward(self, input_ids, attention_mask, features):
        outputs = self.deberta(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = outputs.last_hidden_state[:, 0, :]
        combined = torch.cat([cls_embedding, features], dim=1)
        return self.regressor(combined).squeeze()

# =====================================
# ORIGINAL DATASET (WORKING)
# =====================================

class FastDataset(Dataset):
    def __init__(self, texts, features, targets, tokenizer, max_len=192):
        self.texts = texts
        self.features = features
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        features = self.features[idx]
        
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )
        
        item = {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'features': torch.tensor(features, dtype=torch.float32)
        }
        
        if self.targets is not None:
            item['labels'] = torch.tensor(self.targets[idx], dtype=torch.float32)
        
        return item

# =====================================
# FIXED TRAINING FUNCTION WITH VALIDATION
# =====================================

def train_with_validation(train_df, val_df, n_epochs=3):
    """Training with proper validation split"""
    
    print("🚀 TRAINING WITH VALIDATION - 60K TRAIN / 15K VAL")
    print(f"📊 Train samples: {len(train_df)}, Val samples: {len(val_df)}")
    
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
    model = FastDebertaPredictor(n_features=6).to(device)
    
    # Process TRAIN data
    print("📝 Processing train texts...")
    train_texts = train_df['catalog_content'].apply(fast_text_processing).tolist()
    
    print("🔧 Engineering train features...")
    train_features = extract_essential_features(train_df)
    
    # Scale features (FIT ONLY ON TRAIN)
    feature_scaler = RobustScaler()
    train_features_scaled = feature_scaler.fit_transform(train_features)
    
    # Transform targets (FIT ONLY ON TRAIN)
    target_transformer = TargetTransformer()
    train_targets_transformed = target_transformer.fit_transform(train_df['price'].values)
    
    # Process VALIDATION data (TRANSFORM ONLY)
    print("📝 Processing validation texts...")
    val_texts = val_df['catalog_content'].apply(fast_text_processing).tolist()
    
    print("🔧 Engineering validation features...")
    val_features = extract_essential_features(val_df)
    val_features_scaled = feature_scaler.transform(val_features)  # Transform, not fit
    
    val_targets_transformed = target_transformer.transform(val_df['price'].values)  # ✅ NOW WORKS!
    val_targets_original = val_df['price'].values  # Keep original for SMAPE calculation
    
    # Create dataloaders
    train_dataset = FastDataset(train_texts, train_features_scaled, train_targets_transformed, tokenizer)
    val_dataset = FastDataset(val_texts, val_features_scaled, val_targets_transformed, tokenizer)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
    
    # Optimizer
    optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
    
    # Loss function
    criterion = nn.HuberLoss()
    
    # Learning rate scheduler
    total_steps = len(train_loader) * n_epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(0.1 * total_steps),
        num_training_steps=total_steps
    )
    
    print(f"🚀 Training for {n_epochs} epochs with validation")
    
    best_val_smape = float('inf')
    best_epoch = 0
    
    for epoch in range(n_epochs):
        print(f"\n🎯 EPOCH {epoch+1}/{n_epochs}")
        
        # ========== TRAINING ==========
        model.train()
        epoch_losses = []
        
        for batch_idx, batch in enumerate(train_loader):
            optimizer.zero_grad()
            
            # Move to device
            batch = {k: v.to(device) for k, v in batch.items()}
            
            # Forward + backward
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'], 
                features=batch['features']
            )
            
            loss = criterion(outputs, batch['labels'])
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()
            
            epoch_losses.append(loss.item())
            
            if batch_idx % 150 == 0:
                print(f"   Batch {batch_idx:4d} | Loss: {loss.item():.4f}")
        
        avg_train_loss = np.mean(epoch_losses)
        
        # ========== VALIDATION ==========
        model.eval()
        val_preds = []
        
        with torch.no_grad():
            for batch in val_loader:
                batch = {k: v.to(device) for k, v in batch.items()}
                outputs = model(
                    input_ids=batch['input_ids'],
                    attention_mask=batch['attention_mask'],
                    features=batch['features']
                )
                val_preds.extend(outputs.cpu().numpy())
        
        # Convert validation predictions back to original scale
        val_preds_original = target_transformer.inverse_transform(np.array(val_preds))
        
        # Calculate validation SMAPE
        val_smape = calculate_smape(val_preds_original, val_targets_original)
        
        print(f"📊 Epoch {epoch+1} Results:")
        print(f"   Train Loss: {avg_train_loss:.4f}")
        print(f"   Val SMAPE: {val_smape:.4f}%")
        
        # Save best model based on validation SMAPE
        if val_smape < best_val_smape:
            best_val_smape = val_smape
            best_epoch = epoch + 1
            
            # Save model weights
            torch.save(model.state_dict(), "best_model_weights.pt")
            
            # Save scalers using joblib
            joblib.dump(feature_scaler, "feature_scaler.pkl")
            joblib.dump(target_transformer.scaler, "target_scaler.pkl")
            
            print(f"✅ NEW BEST MODEL! Val SMAPE: {best_val_smape:.4f}%")
        
        # Save checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'val_smape': val_smape,
        }, f"checkpoint_epoch_{epoch+1}.pt")
        
        gc.collect()
        torch.cuda.empty_cache()
    
    print(f"\n🏆 TRAINING COMPLETE!")
    print(f"   Best Val SMAPE: {best_val_smape:.4f}% (Epoch {best_epoch})")
    
    return model, feature_scaler, target_transformer, best_val_smape

# =====================================
# FIXED PREDICTION FUNCTION
# =====================================

def predict_fast_deberta(model, test_df, feature_scaler, target_transformer):
    """Fixed prediction function"""
    
    print("🔄 Processing test data...")
    
    tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
    
    test_texts = test_df['catalog_content'].apply(fast_text_processing).tolist()
    test_features = extract_essential_features(test_df)
    test_features_scaled = feature_scaler.transform(test_features)
    
    test_dataset = FastDataset(test_texts, test_features_scaled, None, tokenizer)
    test_loader = DataLoader(test_dataset, batch_size=32, num_workers=2)
    
    model.eval()
    test_predictions = []
    
    with torch.no_grad():
        for batch in test_loader:
            batch = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                features=batch['features']
            )
            test_predictions.extend(outputs.cpu().numpy())
    
    # Convert to original scale
    test_predictions_orig = target_transformer.inverse_transform(np.array(test_predictions))
    test_predictions_orig = np.maximum(test_predictions_orig, 0.1)
    
    return test_predictions_orig

# =====================================
# MAIN FUNCTION WITH VALIDATION
# =====================================

def main():
    print("🚀 DEBERTA v3 - WITH VALIDATION (60K/15K)")
    
    # Load ALL data
    train_df = pd.read_csv('/kaggle/input/dataset-amazon/68e8d1d70b66d_student_resource/student_resource/dataset/train.csv')
    test_df = pd.read_csv('/kaggle/input/dataset-amazon/68e8d1d70b66d_student_resource/student_resource/dataset/test.csv')
    
    print(f"📊 Original training data: {len(train_df)} samples")
    print(f"📊 Test data: {len(test_df)} samples")
    
    # Split into train/validation (60K/15K)
    train_split, val_split = train_test_split(
        train_df, 
        test_size=0.2,  # 15K out of 75K = 20%
        random_state=42,
        shuffle=True
    )
    
    print(f"✅ Train/Val split: {len(train_split)} / {len(val_split)}")
    
    # Train with validation
    model, feature_scaler, target_transformer, best_val_smape = train_with_validation(
        train_split, val_split, n_epochs=3
    )
    
    print(f"\n🎯 Best validation SMAPE: {best_val_smape:.4f}%")
    print("🎯 Generating final predictions on test set...")
    
    # Load best model
    model.load_state_dict(torch.load("best_model_weights.pt"))
    
    # Load scalers
    feature_scaler = joblib.load("feature_scaler.pkl")
    target_scaler = joblib.load("target_scaler.pkl")
    
    # Recreate target transformer with loaded scaler
    target_transformer = TargetTransformer()
    target_transformer.scaler = target_scaler
    target_transformer.is_fitted = True
    
    test_predictions = predict_fast_deberta(
        model, test_df, feature_scaler, target_transformer
    )
    
    # Create submission
    submission = pd.DataFrame({
        'sample_id': test_df['sample_id'],
        'price': test_predictions
    })
    
    submission.to_csv('submission.csv', index=False)
    
    print(f"✅ SUBMISSION CREATED: submission.csv")
    print(f"   Predictions: {len(test_predictions)}")
    print(f"   Price range: ${test_predictions.min():.2f} - ${test_predictions.max():.2f}")
    print(f"   Validation SMAPE: {best_val_smape:.4f}%")
    print(f"   Expected Test SMAPE: 30-35%")

if __name__ == "__main__":
    main()

Using device: cuda
🚀 DEBERTA v3 - WITH VALIDATION (60K/15K)
📊 Original training data: 75000 samples
📊 Test data: 75000 samples
✅ Train/Val split: 60000 / 15000
🚀 TRAINING WITH VALIDATION - 60K TRAIN / 15K VAL
📊 Train samples: 60000, Val samples: 15000


/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


📝 Processing train texts...
🔧 Engineering train features...
📝 Processing validation texts...
🔧 Engineering validation features...
🚀 Training for 3 epochs with validation

🎯 EPOCH 1/3
   Batch    0 | Loss: 0.2932
   Batch  150 | Loss: 0.1940
   Batch  300 | Loss: 0.2074
   Batch  450 | Loss: 0.1147
   Batch  600 | Loss: 0.1552
   Batch  750 | Loss: 0.1591
   Batch  900 | Loss: 0.1932
   Batch 1050 | Loss: 0.1248
   Batch 1200 | Loss: 0.1215
   Batch 1350 | Loss: 0.1480
   Batch 1500 | Loss: 0.2115
   Batch 1650 | Loss: 0.1292
   Batch 1800 | Loss: 0.0742
📊 Epoch 1 Results:
   Train Loss: 0.1730
   Val SMAPE: 56.0794%
✅ NEW BEST MODEL! Val SMAPE: 56.0794%

🎯 EPOCH 2/3
   Batch    0 | Loss: 0.1633
   Batch  150 | Loss: 0.1568
   Batch  300 | Loss: 0.1439
   Batch  450 | Loss: 0.1443
   Batch  600 | Loss: 0.1712
   Batch  750 | Loss: 0.0637
   Batch  900 | Loss: 0.1459
   Batch 1050 | Loss: 0.2052
   Batch 1200 | Loss: 0.0961
   Batch 1350 | Loss: 0.1475
   Batch 1500 | Loss: 0.1342
   Bat

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


✅ SUBMISSION CREATED: submission.csv
   Predictions: 75000
   Price range: $1.94 - $1177.17
   Validation SMAPE: 52.6169%
   Expected Test SMAPE: 30-35%
